In [1]:
import fsspec
import cf_xarray #noqa
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from kerchunk.netCDF3 import NetCDF3ToZarr
import ujson


fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True)
fs_write = fsspec.filesystem('')

In [2]:
vdatum_files = fs_read.glob('s3://noaa-nos-ofs-pds/OFS_Grid_Datum/*vdatums.nc')

vdatum_files = sorted(['s3://'+f for f in vdatum_files])
vdatum_files

['s3://noaa-nos-ofs-pds/OFS_Grid_Datum/cbofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/ciofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/creofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/dbofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/gomofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/leofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/lmhofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/loofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/lsofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/mbmcf_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/ngofs2_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/nyofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/sfbofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/sjrofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/tbmcf_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/tbofs_vdatums.nc',
 's3://noaa-nos-ofs-pds/OFS_Grid_Datum/wcofs_vdatums.nc']

In [6]:
import ujson

def generate_json(f):
    outname = f'./vdatums/{f.split("/")[-1]}.zarr'

    chunks = NetCDF3ToZarr(f)
    refs = chunks.translate()
    with fs_write.open(outname, 'w') as ofile:
        ofile.write(ujson.dumps(refs))

    return outname

In [7]:
for f in vdatum_files:
    generate_json(f)

In [8]:
# open dataset as zarr object using fsspec reference file system and xarray
fs = fsspec.filesystem("reference", fo='./vdatums/ngofs2_vdatums.nc.zarr', remote_protocol='s3', remote_options={'anon':True})
m = fs.get_mapper("")

ds = xr.open_dataset(m, engine="zarr", chunks={}, backend_kwargs=dict(consolidated=False))
ds

/Users/matthew.iannucci/Developer/ocean-notebooks/venv/lib/python3.10/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:      (N_node: 303714)
Dimensions without coordinates: N_node
Data variables:
    latitude     (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    longitude    (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    mhhwtomsl    (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    mhwtomsl     (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    mllwtomsl    (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    mlwtomsl     (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
    navd88tomsl  (N_node) float64 dask.array<chunksize=(303714,), meta=np.ndarray>
Attributes:
    Conventions:  COARDS
    comment:      Vertical Datum transformations
    file_type:    Full_Grid
    grid_type:    unstructured
    history:      VDATUM Project
    institution:  NOAA/NOS/OCS/CSDL
    model:        Upgraded NGOFS
    references:   http://vdatum.noaa.gov
    source:       CSDL
    title:        Upgraded Northern Gulf of Mexico Operational Forecasting Sy...
    z_type:       sigma